# Part 5 제어 흐름
분기, 루프, 서브루틴 등 순차 제어 흐름을 뛰어넘는 파이썬 언어 기능과 라이브러리에 대해 다룹니다.

# CHPATER 14 반복형, 반복자, 제너레이터

반복자 패턴은 데이터를 처리할 때, 한 번에 하나씩 필요할 때 가져올 수 있게 해줍니다. 이 장에서는 파이썬 어에서 반복자 패턴이 어떻게 구현되어 있는지를 설명합니다.  

모든 제너레이터는 반복자입니다. 제너레이터가 반복자 인터페이스를 구현하고 있기 때문입니다. 하지만, **반복자**는 디자인 패턴에서 정의한 대로 컬렉션에서 항목을 가져오지만, **제너레이터**는 항목을 생성할 수 있습니다.  

파이썬의 컬렉션은 모두 **반복형**입니다. 아래와 같은 연산을 지원하기 위해 내부적으로 반복자를 사용합니다,
* for 루프
* 컬렉션형 생성과 확장
* 텍스트 파일을 한 줄씩 반복
* 지능형 리스트/딕셔너리/집합
* 튜플 언패킹
* 함수 호출 시 * 를 이용해서 실제 매개변수를 언패킹

**이 장에서 다루는 내용**
* 반복형 객체를 처리하기 위해 내부적으로 `iter()` 내장 함수를 사용하는 방법
* 파이썬에서 고전적인 반복자 패턴을 구현하는 방법
* 제너레이터가 작동하는 방식을 한 줄씩 상세히 설명
* 고전적인 반복자를 제너레이터 함수나 제너레이터 표현식으로 바꾸는 방법
* 표준 라이브러리에서의 범용 제너레이터 함수의 활용
* 제너레이터를 결합하기 위해 새로 추가된 `yield from`을 사용하는 방법
* 사례 연구 : 대형 데이터셋을 사용하도록 설계된 데이터베이스 변환 유틸리티에서 제너레이터 함수 이용
* 비슷해 보이는 제너레이터와 코루틴이 실제로는 아주 다르며, 혼합해서 사용하면 안 되는 이유

## 14.1 Sentence 버전 #1: 단어 시퀀스

아래 `Sentence`라는 클래스의 생성자는 텍스트로 구성된 문자열을 받은 후 단어별로 반복한다. #1 버전은 시퀀스 프로토콜을 구현하며 반복한다. 

In [1]:
# 예제 14-1 sentence.py : 단어 시퀀스로서의 Sentence 클래스
import re
import reprlib

RE_WORD = re.compile('\w+') # 문자열을 찾습니다.

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index] 
    # self.words가 위 findall()의 결과를 담고 있으므로, 주어진 인덱스에 해당하는 단어를 반환합니다.
    
    def __len__(self): # 완전한 시퀀스 프로토콜에 따르려면 __len__메서드가 필요. 하지만 반복형 객체에 필요한 것은 아님
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
        # reprlib.repr()은 유틸리티 함수로, 매우 큰 데이터 구조체를 표현하는 문자열을 축약해서 생성(30자로 제한)

In [2]:
# 예제 14-2 Sentence 객체의 반복 텍스트
s = Sentence('"The time has com, " the Walrus said,') # 문자열을 이용해서 Sentence 객체를 생성.
s

Sentence('"The time ha... Walrus said,')

In [3]:
for word in s:
    print(word)

The
time
has
com
the
Walrus
said


In [4]:
list(s)

['The', 'time', 'has', 'com', 'the', 'Walrus', 'said']

위에서 만든 `Sentence` 객체는 반복할 수 있기 때문에 리스트 혹은 다른 반복형을 생성하기 위한 입력으로 사용할 수 있음.

In [5]:
# 예제 14-1에서의 구현은 아래처럼 인덱스를 이용해서 단어를 가져올 수 있다.
s[0]

'The'

In [6]:
s[5]

'Walrus'

In [7]:
s[-1]

'said'

## 14.1.1 Sequence가 반복 가능한 이유 : iter( ) 함수
파이썬 인터프리터가 `x` 객체를 반복해야 할 때는 언제나 `iter(x)`를 자동으로 호출한다.

`iter()`내장 함수는 다음 과정을 수행한다.
1. 객체가 `__iter()__` 메서드를 구현하는지 확인하고, 이 메서드를 호출해서 반복자를 가져온다.
2. `__iter__()` 메서드가 구현되어 있지 않지만 `__getitem__()`이 구현되어 있다면, 파이써은 인덱스 0에서 시작해서 항목을 순서대로 가져오는 반복자를 생성한다.
3. 이 과정이 모두 실패하면 파이썬은 `TypeError: 'C' object is not iterable`이라는 메시지와 함께 `TypeError`가 발생한다. 여기서 `C`는 대상 객체의 클래스다.

시퀀스가 `__getitem__()`을 구현하고 있기 때문에 모든 파이썬 시퀀스는 반복할 수 있다. (2번에 의해서)  
표준 시퀀스는 `__iter__()` 메서드도 구현하고 있으므로 시퀀스를 직접 정의한다면 `__iter__()`를 정의해야 한다.  

In [8]:
class Foo:
    def __iter__(self):
        pass

In [9]:
from collections import abc

In [10]:
issubclass(Foo, abc.Iterable)

True

`issubclass(class, classinfo)` : class가 classinfo의 sub class면 True를 리턴.

In [11]:
f = Foo()

In [12]:
isinstance(f, abc.Iterable)

True

In [13]:
# Sentence 클래스는 반복형으로 사용하고 있지만 iter()메서드가 없으므로 통과하지 못함.
issubclass(Sentence, abc.Iterable)

False

객체를 반복하기 전에 반복형인지 명시적으로 검사할 필요는 없다. 파이썬이 `TypeErorr`를 발생시키기 때문. 예외를 발생시키기 전에 `try/except` 블록으로 처리할 수 있으면 좋음. 반복하기 위해 객체에 저장해두는 경우는 미리 명시적으로 검사해도 좋다.

## 14.2 반복형과 반복자


**반복형**  
`iter()` 내장 함수가 반복자를 가져올 수 있는 모든 객체와 **반복자**를 반환하는 `__iter__()` 메서드를 구현하는 객체는 반복형이다. 0에서 시작하는 인덱스를 받는 `__getitem__()` 메서드를 구현하는 객체인 시퀀스도 마찬가지다.

반복형과 반복자의 관계 : 반복형 객체에서 반복자를 가져온다.

In [14]:
# 'ABC' 문자열은 반복형. 반복자는 내부 어딘가에 있다.
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [15]:
# while 문을 이용하여 작성
s = 'ABC'
it = iter(s) # 반복형으로부터 반복자 it 생성.
while True:
    try:
        print(next(it)) # 반복자에서 next를 호출해 다음 항목을 가져옴.
    except StopIteration: # 반복자 모두 소진
        del it # it에 대한 참조를 해제해서 반복자 객체를 제거함.
        break

A
B
C


반복자에 대한 표준 인터페이스는 아래와 같은 메서드 두 개를 정의한다.  
`__next__()` : 다음에 사용할 항목을 반환, 항목을 모두 소진하면 `Stopiteration`을 발생  
`__iter__()` : `self`를 반환. 그래서 `for` 루프 등 반복형이 필요한 곳에 반복자를 사용할 수 있게 해줌.

![그림14-1](./14-1.png)

`Iterator ABC`는 `return self` 문장만으로 `__iter__()` 메서드를 구현한다. 이렇게 하면 반복형이 필요한 곳에 반복자를 사용할 수 있다.

In [16]:
# 예제 14-13 abc.iterator 클래스의 일부
class Iterator(Iterable):
    
    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        '''반복자에서 다음 항목을 반환한다. 항목이 소진되면 StopIteration 발생'''
        raise StopIteration
        
    def __iter__(self):
        return self
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any("__next__" in B.__dict__ for B in C.__mro__) and
                any("__iter__" in B.__dict__ for B in C.__mro__)):
                return True
        return NotImplemented

NameError: name 'Iterable' is not defined

In [19]:
s3 = Sentence('Pig and Pepper')

In [20]:
it = iter(s3)

In [21]:
it # doctest: +ELLIPSIS

In [22]:
next(it)

'Pig'

In [23]:
next(it)

'and'

In [24]:
next(it)

'Pepper'

In [25]:
next(it)

StopIteration: 

In [26]:
list(it)

[]

In [27]:
list(iter(s3))

['Pig', 'and', 'Pepper']

### 이 장의 정리
**반복자** : 다음 항목을 반환하거나, 다음 항목이 없을 때 `StopIteration` 예외를 발생시키는, 인수를 받지 않는 `__next__()` 메서드를 구현하는 객체. 파이썬 반복자는 `__iter__()` 메서드도 구현하므로 **반복형**이기도 하다.

## 14.3 Sentence 버전 #2: 고전적인 반복자

예제 14-4의 `Sentence` 클래스는 반복형이다. `__iter__()` 메서드를 구현하고 있고, 이 메서드가 `SentenceIterator`를 반환하기 때문. 이 방식이 원래 디자인 패턴에서 설명하고 있는 반복자 디자인 패턴이다.

In [28]:
# 예제 14-4 Sentence_iter.py : 반복자 패턴을 이용한 Sentence 구현
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self): # 반복형 객체를 생성해서 반환함으로써 반복형 프로토콜을 완전히 구현
        return SentenceIterator(self.words)
    
class SentenceIterator: # 단어 리스트에 대한 참조를 담고 있음, 반복자는 __next__()와 __iter__()를 모두 구현해야함
    
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index] # self.index에 있는 단어를 가져옴
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

## 14.3.1 Sentence를 반복자로 만들기: 좋지 않은 생각

반복형과 반복자를 혼동하지 말자.  
반복형은 호출될 때마다 반복자를 새로 생성하는 `__iter__()` 메서드를 가지고 있음.  
반복자는 개별 항목을 반환하는 `__next__()` 메서드와 `self`를 반환하는 `__iter__()` 메서드를 가지고 있음.  
  
따라서, 반복자는 반복형이지만, 반복형은 반복자가 아니다.

위에서 정의한 `Sentence` 클래스 안에 `__iter__()` 외에 `__next__()`도 구현해서 `Sentence` 객체를 반복형이자 반복자로 만들고 싶을 수 있지만 이렇게 해선 안된다!  
  
반복자 패턴은 다음과 같은 용도에 사용하라.
* 집합 객체의 내부 표현을 노출시키지 않고 내용에 접근하는 경우
* 집합 객체의 다중 반복을 지원하는 경우
* 다양한 집합 구조체를 반복하기 위한 통일된 인터페이스를 제공하는 경우  

'다중 반복을 지원'하려면 동일한 반복형 객체로부터 여러 독립적인 반복자를 가질 수 있어야 하며, 각 반복자는 고유한 내부 상태를 유지. 따라서 `iter(my_iterable)`을 호출할 때마다 독립적인 반복자가 새로 만들어져야 한다. 때문에 `SentenceIterator` 클래스가 필요한 것임.

## 14.4 Sentence 버전 #3: 제너레이터 함수

In [29]:
# 예제 14-5 sentence_gen.py : 제너레이터 함수를 사용해서 구현한 Sentence
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word # 현재 단어(word)를 생성
        return # 함수에 끝에 도달하면 값을 자동으로 반환하므로, 이 return문은 필요 없음. 
        # 제너레이터 함수는 StopIteration도 발생시키지 않음. 값을 모두 생산한 후 그냥 빠져나간다.
    
# 완료! 

## 14.4.1 제너레이터 함수의 작동 방식
본체 안에 `yield` 키워드를 가진 함수는 모두 제너레이터 함수. 제너레이터 함수는 호출되면 제너레이터 객체를 반환한다.

In [30]:
def gen_123():
    yield 1
    yield 2
    yield 3
    yield 4    

In [31]:
gen_123 # doctest: +ELLIPSIS

<function __main__.gen_123()>

In [32]:
gen_123() # doctest: +ELLIPSIS

<generator object gen_123 at 0x00000203A2E86AC8>

In [33]:
for i in gen_123():
    print(i)

1
2
3
4


In [34]:
g = gen_123()

In [35]:
next(g)

1

In [36]:
next(g)

2

In [37]:
next(g)

3

In [38]:
next(g)

4

In [39]:
next(g)

StopIteration: 

In [40]:
# 예제 14-6 실행할 때 메시지를 출력하는 제너레이터 함수
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

In [41]:
for c in gen_AB():
    print('-->', c)
    # for 루프에서 첫 next()를 암묵적으로 실행하면 첫 yield 'A'에서 멈춘다. 

start
--> A
continue
--> B
end.


## 14.5 Sentence 버전 #4: 느긋한 구현

느긋한 버전이 왜 필요해요? -> 메모리를 절약하기 위해서입니다.  
  
지금까지 구현한 `Sentence` 버전은 `__iter__()`에서 텍스트 안에 있는 단어의 리스트를 조급하게 생성해서 `self.words` 속성에 바인딩했다. 그러므로 전체 텍스트를 처리하며, 리스트는 거의 텍스트와 맞먹는 양의 메모리를 소비한다.  

In [42]:
# 예제 14-7 sentence_gen2.py : re.finditer() 제너레이터 함수를 호출하는 생성자 함수를 이용해서 구현한 Sentence
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return 'Sentence (%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text): # re.findall()을 쓰지 않고 필요에 따라 match 객체 생성
            yield match.group() # MatchObject 객체에서 매칭되는 텍스트를 추출

## 14.6 Sentence 버전 #5: 제너레이터 표현식

제너레이터 표현식은 지능형 리스트의 느긋한 버전. 필요에 따라 항목을 생성하는 제너레이터를 반환할 수 있기 때문.

In [43]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

In [44]:
# 지능형 리스트는 gen_AB()를 호출해서 생성된 제너레이터 객체가 생성한 항목을 조급하게 반복함.
res1 = [x*3 for x in gen_AB()]

start
continue
end.


In [45]:
for i in res1: # 지능형 리스트가 생성한 res1 리스트를 반복
    print('-->', i)

--> AAA
--> BBB


In [46]:
res2 = (x*3 for x in gen_AB())

In [47]:
res2

<generator object <genexpr> at 0x00000203A42EFD48>

In [48]:
# for 루프가 res2를 반복해야 gen_AB()의 본체가 비로소 실행된다.
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


In [49]:
# 예제 14-9 sentence_genexp.py : 제너레이터 표현식을 사용한 Sentence
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        
    def __repr__(self, text):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text)) # 제너레이터 표현식 사용

## 14.7 제너레이터 표현식: 언제 사용하나?
제너레이터 표현식이 여러 줄에 걸쳐있을 때는 가독성을 위해 제너레이터 함수를 사용.

## 14.8 또 다른 예제: 등차수열 제너레이터

In [50]:
# 예제 14-11 ArithmeticProgression 클래스

class ArithmeticProgression:
    
    def __init__(self, begin, step, end=None): # end는 선택인수
        self.begin = begin 
        self.step = step
        self.end = end # None이면 무한 수열. 
        
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin) # 이후에 더할 값에 맞춰 자료형 강제 변환
        forever = self.end is None # forever 플래그
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index # 다음에 가져올 result 미리 계싼. 

In [51]:
# 예제 14-10 ArithmeticProgression 사용 예
ap = ArithmeticProgression(0, 1, 3)

In [52]:
list(ap)

[0, 1, 2]

In [53]:
ap = ArithmeticProgression(0, 1/3, 1)

In [54]:
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [55]:
from fractions import Fraction

In [56]:
ap = ArithmeticProgression(0, Fraction(1, 3), 1)

In [57]:
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [58]:
from decimal import Decimal

In [59]:
ap = ArithmeticProgression(0, Decimal('.1'), .3)

In [60]:
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

In [61]:
# 예제 14-12 atitprog_gen() 제너레이터 함수
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

## 14.8.1 itertools를 이용한 등차수열

`itertools.count()` 함수는 숫자를 생성하는 제너레이터를 반환한다. 인수를 지정하지 않으면 0에서 시작하는 수열을 생성.  
그러나 `start`와 `stop` 인수를 지정하면 앞에서 구현한 `aritprog_gen()` 함수와 아주 비슷한 결과를 낼 수 있다.

In [62]:
import itertools

In [63]:
gen = itertools.count(1, .5)

In [64]:
next(gen)

1

In [65]:
next(gen)

1.5

In [66]:
next(gen)

2.0

In [67]:
next(gen)

2.5

`itertools.count()`는 끝이 없다.. 따라서 `list(count())`를 실행하면 실패함.  
`itertools.takewhile()`이라는 함수는 다른 제너레이터를 소비하면서 주어진 조건식(predicate)이 `False`가 되면 중단되는 제너레이터를 생성한다.

In [68]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))

In [69]:
list(gen)

[1, 1.5, 2.0, 2.5]

In [70]:
# 예제 14-13 aritprog_v3.py : 이전 aritprog_gen() 함수와 동일하게 작동
import itertools

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

## 14.9 표준 라이브러리의 제너레이터 함수

### 필터링 제너레이터

In [71]:
# 예제 14-14 필터링 제너레이터 함수 예
def vowel(c):
    return c.lower() in 'aeiou'

In [72]:
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [73]:
import itertools

`filterfalse(predicate, it)` : `filter()`와 같지만 반대 논리를 적용. `predicate`로 거짓된 값이 나오는 항목 모두 생성

In [74]:
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

`dropwhile(predicate, it)` : `predicate`가 참된 값인 동안 항목들을 지나가면서 `it`를 소비한 후, 추가 검사 없이 남아 있는 항목을 모두 생성

In [75]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

`compress(it, selector_it)` : 두 개의 반복형을 병렬로 소비. `selector_it`의 해당 항목이 참된 값일 때마다 `it`에서 항목을 생성

In [76]:
list(itertools.compress('Aardvark', (1,0,1,1,0,1)))

['A', 'r', 'd', 'a']

`islice(it.stop)` or `islice(it, start, stop, step=1)` : s[:stop]이나 s[start:stop:step]과 비슷하게, 반복할 수 있는 모든 객체에서 느긋하게 연산을 적용해서 `it`의 슬라이스 항목을 생성

In [77]:
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [78]:
list(itertools.islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [79]:
list(itertools.islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']

### 매핑 제너레이터

In [80]:
# 예제 14-15 itertools.accumulate() 제너레이터 함수 예
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
import itertools
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [81]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [82]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [83]:
import operator

In [84]:
list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [85]:
list(itertools.accumulate(range(1, 11), operator.mul))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [86]:
# 예제 14-16 매핑 제너레이터 함수 예
list(enumerate('albatroz', 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [87]:
import operator

In [88]:
# 정수의 제곱을 구한다.
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [89]:
# 나란히 가져온 숫자를 곱한다. range(11)에서 0, 1, 2 밖에 안 빼냄
list(map(operator.mul, range(11), [2, 4, 8]))

[0, 4, 16]

In [90]:
# zip() 내장 함수의 연산을 모방한다.
list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))

[(0, 2), (1, 4), (2, 8)]

In [91]:
# 1부터 시작해서 단어 안의 각 글자를 인덱스만큼 반복한다.
import itertools
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [92]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

In [93]:
# 이동 평균 구하기
list(itertools.starmap(lambda a, b: b/a,
                       enumerate(itertools.accumulate(sample), 1)))

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

### 병합 제너레이터 함수

In [94]:
# 예제 14-17 병합 생성자 함수 예

# 일반적으로 chain()은 두 개 이상의 반복형을 전달해서 호출한다.
list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [95]:
# 반복형을 하나만 입력받으면, chain()은 별로 유용한 작업을 수행하지 않는다.
list(itertools.chain(enumerate('ABC')))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [96]:
# chain.from_iterable()은 반복형에서 항목을 하나씩 가져와서, 각 항목이 반복형이면 시퀀스 안에 연결한다.
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [97]:
# 일반적으로 zip()은 두 개의 반복형을 이용해서 두 항목으로 구성된 튜플의 열을 만든다.
list(zip('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [98]:
# zip()은 여러 개의 반복형도 병렬로 소비할 수 있지만, 가장 짧은 반복형이 끝나자마자 생성을 중단한다.
list(zip('ABC', range(5), [10, 20, 30, 40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [99]:
# itertools.zip_longest()는 zip()과 비슷하지만, 필요에 따라 튜플을 None으로 채워가며 모든 반복형을 소모할 때 까지 생성한다.
list(itertools.zip_longest('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [100]:
# fillvalue 키워드 인수를 이용해서 빈 항목을 채우는 값으로 사용한다. 
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

In [101]:
# 예제 14-18 itertools.product() 제너레이터 함수 예

In [102]:
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [103]:
suits = 'spades hearts diamonds clubs'.split()

In [104]:
list(itertools.product('AK', suits))

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [105]:
list(itertools.product('ABC'))

[('A',), ('B',), ('C',)]

In [106]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [107]:
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [108]:
rows = itertools.product('AB', range(2), repeat=2)

In [109]:
for row in rows: print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


In [110]:
# 예제 14-19 count(), cycle(), repeat() 사용 예

In [111]:
ct = itertools.count()

In [112]:
next(ct)

0

In [113]:
next(ct), next(ct), next(ct)

(1, 2, 3)

In [114]:
list(itertools.islice(itertools.count(1, .3), 3))

[1, 1.3, 1.6]

In [115]:
cy = itertools.cycle('ABC')

In [116]:
next(cy)

'A'

In [117]:
list(itertools.islice(cy, 7))

['B', 'C', 'A', 'B', 'C', 'A', 'B']

In [118]:
rp = itertools.repeat(7)

In [119]:
next(rp), next(rp)

(7, 7)

In [120]:
list(itertools.repeat(8, 4))

[8, 8, 8, 8]

In [121]:
list(map(operator.mul, range(11), itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [122]:
# 예제 14-20 하나의 입력 항목마다 값을 여러 개 생성하는 순열 조합 제너레이터 함수
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [123]:
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [124]:
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [125]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [126]:
# 예제 14-21 itertools.groupby()와 내장 함수 reversed()의 사용 예
list(itertools.groupby('LLLLAAGGG'))

[('L', <itertools._grouper at 0x203a432f648>),
 ('A', <itertools._grouper at 0x203a432fdc8>),
 ('G', <itertools._grouper at 0x203a432f188>)]

In [127]:
for char, group in itertools.groupby('LLLLAAAGG'):
    print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A', 'A']
G -> ['G', 'G']


In [128]:
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat', 'dolphin', 'shark', 'lion']

In [129]:
animals.sort(key=len)

In [130]:
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

In [131]:
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']


In [132]:
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


In [133]:
# 예제 14-22 itertools.tee()는 여러 개의 제너레이터를 생성하며, 각 제너레이터는 입력된 내용을 그대로 생성한다.
list(itertools.tee('ABC'))

[<itertools._tee at 0x203a42f9e48>, <itertools._tee at 0x203a43520c8>]

In [134]:
g1, g2 = itertools.tee('ABC')

In [135]:
next(g1)

'A'

In [136]:
next(g2)

'A'

In [137]:
next(g2)

'B'

In [138]:
list(g1)

['B', 'C']

In [139]:
list(g2)

['C']

In [140]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

## 14.10 파이썬 3.3의 새로운 구문: yield from

In [141]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

In [142]:
s = 'ABC'

In [143]:
t = tuple(range(3))

In [144]:
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [145]:
def chain(*iterables):
    for i in iterables:
        yield from i # for문을 완전히 대체함. 

In [146]:
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

## 14.11 반복형을 리듀스하는 함수

[표 14-6]의 함수는 모두 반복형을 입력받아 하나의 값을 반환하는 '리듀스', '폴딩', '누적' 함수.

![표14-6](./14-2.png)

In [147]:
# 예제 14-23 어떤 시퀀스에 all()과 any()를 적용한 결과
all([1, 2, 3])

True

In [148]:
all([1, 0, 3])

False

In [149]:
all([])

True

In [150]:
any([1, 2, 3])

True

In [151]:
any([1, 0, 3])

True

In [152]:
any([0, 0.0])

False

In [153]:
any([])

False

In [154]:
g = (n for n in [0, 0.0, 7, 8])

In [155]:
any(g)

True

In [156]:
next(g)

8

## 14.12 iter() 함수 들여다보기

`iter()` 함수를 일반 함수나 콜러블 객체로부터 반복자를 생성하기 위해 두 개의 인수를 전달해서 호출할 수 있다. 이렇게 사용하려면, 첫 번째 인수는 값을 생성하기 위해 인수없이 반복적으로 호출되는 콜러블이어야 하며, 두번째 인수는 구분 표시(sentinel)로서, 콜러블에서 이 값이 반환되면 반복자가 `StopIteration` 예외를 발생시키도록 만든다.

In [157]:
from random import randint

In [158]:
# 1이 나올 때까지 주사위를 굴려보자
def d6():
    return randint(1, 6)

In [159]:
d6_iter = iter(d6, 1)

In [160]:
d6_iter

In [161]:
for roll in d6_iter:
    print(roll)

2


여기서 `iter()` 함수가 `callable_iterator` 객체를 반환함에 주의. 예제 안에 있는 for 루프는 아무리 오래 실행하더라도 결코 1을 출력하지 않음. 1이 구분 표시이기 때문이다. 반복자와 마찬가지로 `d6_iter` 객체는 일단 소모하고 난 후에는 쓸모가 없어진다. 다시 시작하려면 `iter()` 함수를 한 번 더 호출해서 반복자를 다시 만들어야 한다.

In [162]:
# 빈 줄을 발견하거나 파일의 끝에 도달할 때까지 한 줄씩 읽어서 처리
with open('mydata.txt') as fp:
    for lin in iter(fp.readline, ''):
        process_line(line)

FileNotFoundError: [Errno 2] No such file or directory: 'mydata.txt'

## 14.13 사례 연구: 데이터베이스 변환 유틸리티 안의 제너레이터

대충 하나의 for 루프 안에서 두 개의 다른 API로 파일을 읽어야 하는데, 이 때 제너레이터 함수를 두 개를 읽어 처리함.

## 14.14 코루틴으로서의 제너레이터

`send()` : 제너레이터가 다음 `yield`로 넘어가게 만들지만, 제너레이터를 사용하는 호출자가 제너레이터에 데이터를 보낼 수도 있게 해준다. `send()`에 전달된 인수는 모두 제너레이터 함수 본체 안에서 해당 `yield` 표현식의 값이 된다. 즉, 호출자가 제너레이터로부터 데이터를 받는 것만 허용하는 `__next()__`와 달리, `send()` 메서드는 호출자와 제너레이터가 양방향으로 데이터를 교환할 수 있게 해준다.

## 14.15 요약

* `Sentence` 클래스에 제너레이터를 사용하여 간결하고 가독성있게 구성했다.
* 등차수열 제너레이터를 구현해봤다.
* `itertools` 모듈 사용법을 익혔다.
* `iter(func, sentinel)` 내장 함수를 사용했다.

## 14.16 읽을거리

### 제너레이터와 반복자의 의미

반복자와 제너레이터의 관계는 세 가지 측면에서 바라볼 수 있다.
1. 인터페이스 관점  
파이썬 반복자 프로토콜은 `__next__()`와 `__iter__()` 메서드를 정의한다. 제너레이터 객체는 이 메서드를 모두 구현하므로, 이런 관점에서 보면 제너레이터는 일종의 반복자다. 이 정의에 따르면 `enumerate()` 내장 함수로 생성한 객체는 반복자다.

In [163]:
from collections import abc
e = enumerate('ABC')
isinstance(e, abc.Iterator)

True

2. 구현 관점  
    * 이 관점에서 보면, 제너레이터는 `yield` 키워드를 가진 함수나 제너레이터 표현식으로 구현할 수 있는 파이썬 언어 구성체.
    * 제너레이터 함수를 호출 또는 제너레이터 표현식을 평가해서 생성된 제너레이터 객체는 내부적으로는 `GeneratorType` 객체로 다뤄짐.
    * 이 관점에서 보면 모든 제너레이터는 반복자.
    * 하지만, 예제 14-4(Sentence 클래스 - getitem없이 만든 고전적인 반복자)처럼 제너레이터가 아닌 반복자도 만들 수 있다.
    * 이 관점에서는 `enumerate` 객체는 제너레이터가 아니다.

In [164]:
import types
e = enumerate('ABC')
isinstance(e, types.GeneratorType)

False

3. 개념적 관점
    * 디자인 패턴에 정의한 고전적인 반복자 디자인 패턴에서는 반복자 컬렉션을 순회하면서 항목을 생성한다고 정의.
    * 반복자는 기존 데이터를 읽고, `next(it)`가 호출되면 읽은 값을 변경하지 않고 그대로 생성해야 함.
    * 제너레이터는 `range()`와 마찬가지로 컬렉션을 순회하지 않고도 값을 생성할 수 있다.
    * 컬렉션에 연결되어 있는 경우에도, 그 안에 들어있는 항목을 그대로 사용하지 않고 유도한 값을 생성할 수 있음.
    * `enumerate()`가 대표적인 제너레이터(튜플을 생성하므로 반복자가 아님!)

In [165]:
# 예제 14-26 fibo_by_hand.py: GeneratorType 객체를 사용하지 않고 구현한 피보나치 수열 제너레이터
class Fibonacci:
    
    def __iter__(self):
        return FibonacciGenerator()
    
class FibonacciGenerator:
    
    def __init__(self):
        self.a = 0
        self.b = 1
        
    def __next__(self):
        result = self.a
        self.a, self.b = self.b, self.a + self.b
        return result
    
    def __iter__(self):
        return self

In [166]:
# list(f)

In [167]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [173]:
# 위에껀 무한 피보나치 수열...
def fibonacci(end):
    a, b = 0, 1
    while a < end:
        yield a
        a, b = b, a + b

In [181]:
a = fibonacci(10)

In [182]:
list(a)

[0, 1, 1, 2, 3, 5, 8]